In [1]:
from transformers import SpeechT5Processor, SpeechT5ForSpeechToText, SpeechT5Tokenizer, SpeechT5Model, SpeechT5FeatureExtractor
from datasets import load_dataset

import torch

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Data

In [3]:
dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("id")
example = dataset[40]

Found cached dataset librispeech_asr_demo (/export/home/lium/bdos/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr_demo/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b)


## Model

In [4]:
#processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_asr")

In [5]:
feature_extractor = SpeechT5FeatureExtractor.from_pretrained("microsoft/speecht5_asr")
feature_extractor

SpeechT5FeatureExtractor {
  "do_normalize": false,
  "feature_extractor_type": "SpeechT5FeatureExtractor",
  "feature_size": 1,
  "fmax": 7600,
  "fmin": 80,
  "frame_signal_scale": 1.0,
  "hop_length": 16,
  "mel_floor": 1e-10,
  "num_mel_bins": 80,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "SpeechT5Processor",
  "reduction_factor": 2,
  "return_attention_mask": true,
  "sampling_rate": 16000,
  "win_function": "hann_window",
  "win_length": 64
}

In [6]:
#feature_extractor.do_normalize = True
#feature_extractor.feature_size = 2

In [7]:
#feature_extractor.do_normalize

In [17]:
stt = SpeechT5ForSpeechToText.from_pretrained("microsoft/speecht5_asr")
stt = stt.to(device)
model = SpeechT5Model.from_pretrained("microsoft/speecht5_asr")
model = model.to(device)

Some weights of the model checkpoint at microsoft/speecht5_asr were not used when initializing SpeechT5Model: ['text_decoder_postnet.lm_head.weight', 'speecht5.decoder.prenet.embed_tokens.weight', 'speecht5.encoder.prenet.feature_encoder.conv_layers.5.conv.weight', 'speecht5.encoder.prenet.feature_encoder.conv_layers.0.layer_norm.bias', 'speecht5.encoder.prenet.feature_projection.layer_norm.bias', 'speecht5.encoder.prenet.feature_encoder.conv_layers.0.conv.weight', 'speecht5.decoder.prenet.embed_positions.weights', 'speecht5.encoder.prenet.feature_encoder.conv_layers.2.conv.weight', 'speecht5.encoder.prenet.pos_conv_embed.conv.weight_g', 'speecht5.encoder.prenet.pos_conv_embed.conv.bias', 'speecht5.encoder.prenet.feature_encoder.conv_layers.3.conv.weight', 'speecht5.encoder.prenet.feature_projection.layer_norm.weight', 'speecht5.encoder.prenet.feature_encoder.conv_layers.4.conv.weight', 'speecht5.encoder.prenet.masked_spec_embed', 'speecht5.encoder.prenet.feature_encoder.conv_layers.6.

SpeechT5ForSpeechToText(
  (speecht5): SpeechT5Model(
    (encoder): SpeechT5EncoderWithSpeechPrenet(
      (prenet): SpeechT5SpeechEncoderPrenet(
        (feature_encoder): SpeechT5FeatureEncoder(
          (conv_layers): ModuleList(
            (0): SpeechT5GroupNormConvLayer(
              (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
              (activation): GELUActivation()
              (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
            )
            (1-4): 4 x SpeechT5NoLayerNormConvLayer(
              (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
              (activation): GELUActivation()
            )
            (5-6): 2 x SpeechT5NoLayerNormConvLayer(
              (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
              (activation): GELUActivation()
            )
          )
        )
        (feature_projection): SpeechT5FeatureProjection(
          (layer_norm): LayerNorm((512,),

In [9]:
sampling_rate = dataset.features["audio"].sampling_rate
#inputs = processor(audio=example["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt").to(device)

inputs2 = feature_extractor(example["audio"]["array"], return_tensors="pt", sampling_rate=sampling_rate, truncation=True, max_length=768).to(device)

In [10]:
sampling_rate

16000

In [12]:
inputs2

{'input_values': tensor([[-4.8828e-04, -1.8311e-04, -1.3733e-03, -1.2207e-04, -7.3242e-04,
          4.2725e-04,  1.2817e-03,  1.8616e-03,  3.0212e-03, -4.2725e-04,
         -4.2725e-04,  9.1553e-04, -1.8311e-04, -1.5869e-03,  7.3242e-04,
          4.1809e-03,  1.4343e-03, -6.7139e-04,  2.4414e-04, -4.5776e-04,
         -1.1597e-03,  1.5259e-04,  2.5940e-03, -3.9673e-04, -2.2583e-03,
          5.1880e-04,  8.8501e-04, -1.1292e-03, -1.6479e-03,  2.5635e-03,
          2.4109e-03, -1.6785e-03, -2.1057e-03, -1.8311e-04,  8.5449e-04,
          8.8501e-04,  3.7842e-03,  1.7395e-03, -8.8501e-04, -3.9673e-04,
          3.0518e-04, -2.4414e-04, -9.1553e-05,  2.7466e-03,  2.1667e-03,
         -9.1553e-05,  9.7656e-04,  1.1902e-03,  2.4414e-04,  6.4087e-04,
          8.2397e-04, -1.3428e-03, -3.4180e-03, -1.6479e-03,  6.4087e-04,
          6.4087e-04,  8.8501e-04,  2.2278e-03,  2.2278e-03,  6.1035e-05,
         -1.4038e-03, -5.4932e-04, -1.1902e-03, -2.3193e-03, -6.7139e-04,
          1.2207e-03,

In [13]:
#inputs.input_values.size()

In [14]:
inputs2.input_values.size()

torch.Size([1, 768])

In [24]:
tokenizer = SpeechT5Tokenizer.from_pretrained("microsoft/speecht5_asr")

In [25]:
tokenizer

SpeechT5Tokenizer(name_or_path='microsoft/speecht5_asr', vocab_size=79, model_max_length=450, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)

## Speech Embeddings

In [23]:
# Compute the embeddings
with torch.no_grad():
    out = stt.forward(inputs2.input_values)

ValueError: You have to specify `decoder_input_ids`

In [ ]:
# Compute the embeddings
#with torch.no_grad():
#    embeddings = model(**inputs).last_hidden_state